In [ ]:
# # This file is part of Theano Geometry
#
# Copyright (C) 2017, Stefan Sommer (sommer@di.ku.dk)
# https://bitbucket.org/stefansommer/theanogemetry
#
# Theano Geometry is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# Theano Geometry is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Theano Geometry. If not, see <http://www.gnu.org/licenses/>.
#

# Frame Bundle Geometry on Embedded Ellipsoids

In [ ]:
from src.manifolds.ellipsoid import *
M = Ellipsoid(params=np.array([1,1,1]))
#M.params.set_value(np.array([1,0.5,0.5]))
print(M)
from src.plotting import *

In [ ]:
# Riemannian structure
from src.Riemannian import metric
metric.initialize(M,truncate_high_order_derivatives=True)

# geodesics
from src.Riemannian import geodesic
geodesic.initialize(M)

In [ ]:
# frame bundle
from src.framebundle import FM
FM.initialize(M)

In [ ]:
# test that adapated bases D and D^* are dual
x1 = M.coordsf([1.2,0.])
nu1 = np.dot(np.diag((.5,1.)),np.linalg.cholesky(M.gsharpf(x1)))
u1 = (np.concatenate((x1[0],nu1.flatten())),x1[1])

A = np.zeros((6,6))
for i in range(6):
    Dp1 = np.zeros(6)
    Dp1[i] = 1.
    p1 = M.from_Dstarf(u1,Dp1)
    
    for j in range(6):
        Dv1 = np.zeros(6)
        Dv1[j] = 1.
        v1 = M.from_Df(u1,Dv1)

        A[i,j] = np.dot(v1,p1)
print(A)

A = np.zeros((6,6))
for i in range(6):
    p1 = np.zeros(6)
    p1[i] = 1.
    Dp1 = M.to_Dstarf(u1,p1)
    
    for j in range(6):
        v1 = np.zeros(6)
        v1[j] = 1.
        Dv1 = M.to_Df(u1,v1)

        A[i,j] = np.dot(Dv1,Dp1)
print(A)

In [ ]:
# elements
x = M.coordsf([0.,0.])

# element u=(x,nu) in FM, nu being frame for T_xM
# np.linalg.cholesky(M.gsharpf(x)) gives orthonormal basis for T_xM, multiplication scales in given directions
nu = np.dot(np.diag((.5,1.)),np.linalg.cholesky(M.gsharpf(x)))
u = (np.concatenate((x[0],nu.flatten())),x[1])

# FM covector p
v = tensor([2.,2.])
px = np.linalg.solve(nu,v) # manifold part
pu = tensor([0.,0.,0.,0.]) # frame part
p = np.concatenate([px,pu])

print("u = ", u)
print("p = ", p)

newfig()
M.plot()
M.plotx(x,v=nu)

## FM Geodesics

In [ ]:
# Hamiltionian dynamics on FM from sub-Riemannian structure <v,w>_FM=<u^-1(v),u^-1(w)>_R^2
from src.framebundle import Hamiltonian_FM
Hamiltonian_FM.initialize(M)

In [ ]:
# test that chart update preserves FM Hamiltonian
x1 = M.coordsf([1.2,0.])
nu1 = np.dot(np.diag((.5,1.)),np.linalg.cholesky(M.gsharpf(x1)))
u1 = (np.concatenate((x1[0],nu1.flatten())),x1[1])

for i in range(6):
#     Dp1 = np.zeros(6)
#     Dp1[i] = 1.
#     p1 = M.from_Dstarf(u1,Dp1)
    p1 = np.zeros(6)
    p1[i] = 1.

    print("M.H_FM x1:",M.H_FMf(u1,p1))
    chart2 = M.centered_chartf(M.Ff(x1))
    up2 = M.chart_update_Hamiltonian_FMf(u1,p1)
    print("M.H_FM x2:",M.H_FMf((up2[0],chart2),up2[1]))

In [ ]:
print(M.H_FMf(u,p))

# compute FM geodesic
(us,charts) = M.Exp_Hamiltonian_FMtf(u,p)

# plot
newfig()
M.plot(rotate=(30,80))
M.plot_path(zip(us,charts),v_steps=np.arange(0,n_steps.eval(),5),linewidth=1.5,s=50)
plt.show()

# dynamics returning both position and momentum
(ts,qps,charts) = M.Hamiltonian_dynamics_FMf(u,p)
us = qps[:,0,:]
ps = qps[:,1,:]
print("Energy: ",np.array([M.H_FMf((q,charts),p) for (q,p,charts) in zip(us,ps,charts)]))

# Development and Stochastic Development

In [ ]:
# development dynamics
from src.stochastics import stochastic_development
stochastic_development.initialize(M)

In [ ]:
# deterministic development

# curve in R^2
t = np.linspace(0,10,n_steps.get_value()+1)
gamma = np.vstack([[20*np.sin(t), t**2 + 2*t]]).T
dgamma = np.diff(gamma, axis = 0)

(ts,us,charts) = M.developmentf(u,dgamma)

# plot with frame
newfig()
M.plot()
M.plot_path(zip(us,charts),v_steps=np.arange(0,n_steps.eval(),5))
plt.show()

# plot only trajectory
newfig()
M.plot()
M.plot_path(zip(us[:,0:M.dim.eval()],charts))
plt.show()

# plot anti-development
plt.figure()
plt.plot(gamma[:,0],gamma[:,1])
plt.axis('equal')
plt.show()

In [ ]:
n_steps.set_value(1000)

# stochastic development
w = dWsf(M.dim.eval()) # noise / anti-development
(ts,us,charts) = M.stochastic_developmentf(u,w)

# plot with frame
newfig()
M.plot()
M.plot_path(zip(us,charts),v_steps=np.arange(0,n_steps.eval(),50))
plt.show()

# plot only trajectory
newfig()
M.plot()
M.plot_path(zip(us[:,0:M.dim.eval()],charts))
plt.show()

# plot noise / anti-development
plt.figure()
ws = np.cumsum(w,axis=0)
plt.plot(ws[:,0],ws[:,1])
plt.axis('equal')
plt.show()

n_steps.set_value(100)

In [ ]:
# # This file is part of Theano Geometry
#
# Copyright (C) 2017, Stefan Sommer (sommer@di.ku.dk)
# https://bitbucket.org/stefansommer/theanogemetry
#
# Theano Geometry is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# Theano Geometry is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Theano Geometry. If not, see <http://www.gnu.org/licenses/>.
#

from src.utils import *
import src.linalg as linalg

#######################################################################
# guided processes, Delyon/Hu 2006                                    #
#######################################################################

# hit target v at time t=Tend
def get_sde_guided(M, sde_f, phi, sqrtCov, A=None, method='DelyonHu', integration='ito', use_charts=False, chart_update=None, v_chart_update=None):
    assert (integration == 'ito' or integration == 'stratonovich')
    assert (method == 'DelyonHu')  # more general schemes not implemented
    
    def chart_update_guided(t, x, chart, log_likelihood, log_varphi, h, v, *ys):
        if chart_update is None:
            return (t, x, chart, log_likelihood, log_varphi, h, v, *ys)

        (t_new, x_new, chart_new, *ys_new) = chart_update(t,x,chart,*ys)
        v_new = v if v_chart_update is None else M.update_coords((v,chart),chart_new)[0]
        return (t_new, x_new, chart_new, log_likelihood, log_varphi, h, v_new, *ys_new)

    def sde_guided(dW, t, x, chart, log_likelihood, log_varphi, h, v, *ys):
        if not use_charts:
            (det, sto, X, *dys_sde) = sde_f(dW, t, x, *ys)
        else:
            (det, sto, X, *dys_sde) = sde_f(dW, t, x, chart, *ys)
            
        xchart = x if not use_charts else (x,chart)

        h = theano.ifelse.ifelse(T.lt(t, Tend - dt / 2),
                                 phi(xchart, v) / (Tend - t),
                                 T.zeros_like(phi(xchart, v))
                                 )
        sto = theano.ifelse.ifelse(T.lt(t, Tend - 3 * dt / 2),  # for Ito as well?
                                   sto,
                                   T.zeros_like(sto)
                                   )

        ### likelihood
        dW_guided = (1 - .5 * dt / (1 - t)) * dW + dt * h  # for Ito as well?
        sqrtCovx = sqrtCov(xchart)
        Cov = dt * T.tensordot(sqrtCovx, sqrtCovx, (1, 1))
        Pres = T.nlinalg.MatrixInverse()(Cov)
        residual = T.tensordot(dW_guided, T.tensordot(Pres, dW_guided, (1, 0)), (0, 0))
        log_likelihood = .5 * (-dW.shape[0] * T.log(2 * np.pi) + linalg.LogAbsDet()(Pres) - residual)

        ## correction factor
        ytilde = T.tensordot(X, h * (Tend - t), 1)
        tp1 = t + dt
        if integration == 'ito':
            xtp1 = x + dt * (det + T.tensordot(X, h, 1)) + sto
        elif integration == 'stratonovich':
            tx = x + sto
            xtp1 = x + dt * det + 0.5 * (sto + sde_f(dW, tp1, tx, *ys)[1])
        xtp1chart = xtp1 if not use_charts else (xtp1,chart)
        if not use_charts:
            Xtp1 = sde_f(dW, tp1, xtp1, *ys)[2]
        else:
            Xtp1 = sde_f(dW, tp1, xtp1, chart, *ys)[2]
        ytildetp1 = T.tensordot(Xtp1, phi(xtp1chart, v), 1)

        # set default A if not specified
        Af = A if A is not None else lambda x, v, w: T.tensordot(v, T.tensordot(T.nlinalg.MatrixInverse()(T.tensordot(X, X, (1, 1))), w, 1), 1)

        #     add t1 term for general phi
        #     dxbdxt = theano.gradient.Rop((Gx-x[0]).flatten(),x[0],dx[0]) # use this for general phi
        t2 = theano.ifelse.ifelse(T.lt(t, Tend - dt / 2),
                                  -Af(xchart, ytilde, dt * det) / (Tend - t),
                                  # check det term for Stratonovich (correction likely missing)
                                  constant(0.))
        t34 = theano.ifelse.ifelse(T.lt(tp1, Tend - dt / 2),
                                   -(Af(xtp1chart, ytildetp1, ytildetp1) - Af(xchart, ytildetp1, ytildetp1)) / (
                                   2 * (Tend - tp1 + dt * T.gt(tp1, Tend - dt / 2))),
                                   # last term in divison is to avoid NaN with non-lazy Theano conditional evaluation
                                   constant(0.))
        log_varphi = t2 + t34

        return (det + T.tensordot(X, h, 1), sto, X, log_likelihood, log_varphi, dW_guided/dt, T.zeros_like(v), *dys_sde)

    if not use_charts:
        return lambda dW, t, x, log_likelihood, log_varphi, h, v, *ys: sde_guided(dW, t, x, None, log_likelihood, log_varphi, h, v, *ys)
    else:
        return (sde_guided, chart_update_guided)

def get_guided_likelihood(M, sde_f, phi, sqrtCov, A=None, method='DelyonHu', integration='ito', use_charts=False, chart_update=None):
    v = M.sym_element()
    if not use_charts:
        sde_guided = get_sde_guided(M, sde_f, phi, sqrtCov, A, method, integration)
        guided = lambda q, v, dWt: integrate_sde(sde_guided,
                                                 integrator_ito if method == 'ito' else integrator_stratonovich,
                                                 None,
                                                 q, None, dWt, constant(0.), constant(0.), T.zeros_like(dWt[0]), v)
        guidedf = M.function(guided,v,dWt)
    else:
        (sde_guided,chart_update_guided) = get_sde_guided(M, sde_f, phi, sqrtCov, A, method, integration, use_charts=True, chart_update=chart_update)
        guided = lambda q, v, dWt: integrate_sde(sde_guided,
                                                 integrator_ito if method == 'ito' else integrator_stratonovich,
                                                 chart_update_guided,
                                                 q[0], q[1], dWt, constant(0.), constant(0.), T.zeros_like(dWt[0]), v)
        guidedf = M.coords_function(guided,v,dWt)

    return (guided, guidedf)

import src.linalg as linalg

def bridge_sampling(lg,bridge_sdef,dWsf,options,pars):
    """ sample samples_per_obs bridges """
    (v,seed) = pars
    if seed:
        srng.seed(seed)
    bridges = np.zeros((options['samples_per_obs'],n_steps.eval(),)+lg.shape)
    log_varphis = np.zeros((options['samples_per_obs'],))
    log_likelihoods = np.zeros((options['samples_per_obs'],))
    for i in range(options['samples_per_obs']):
        (ts,gs,log_likelihood,log_varphi) = bridge_sdef(lg,v,dWsf())[:4]
        bridges[i] = gs
        log_varphis[i] = log_varphi[-1]
        log_likelihoods[i] = log_likelihood[-1]
        try:
            v = options['update_vf'](v) # update v, e.g. simulate in fiber
        except KeyError:
            pass
    return (bridges,log_varphis,log_likelihoods,v)

# helper for log-transition density
def p_T_log_p_T(g, v, dWs, bridge_sde, phi, options, F=None, sde=None, use_charts=False, chain_sampler=None, init_chain=None):
    """ Monte Carlo approximation of log transition density from guided process """
    if use_charts:
        chart = g[1]
    
    # sample noise
    (cout, updates) = theano.scan(fn=lambda x: dWs,
                                  outputs_info=[T.zeros_like(dWs)],
                                  n_steps=options['samples_per_obs'])
    dWsi = cout
    
    # map v to M
    if F is not None:
        v = F(v if not use_charts else (v,chart))

    if not 'update_v' in options:
        # v constant throughout sampling
        print("transition density with v constant")
        
        # bridges
        Cgv = T.sum(phi(g, v) ** 2)
        def bridge_logvarphis(dWs, log_varphi, chain):
            if chain_sampler is None:
                w = dWs
            else:
                (accept,new_w) = chain_sampler(chain)
                w = T.switch(accept,new_w,w)
            if not use_charts:
                (ts, gs, log_likelihood, log_varphi) = bridge_sde(g, v, theano.gradient.disconnected_grad(w))[:4] # we don't take gradients of the sampling scheme
            else:
                (ts, gs, charts, log_likelihood, log_varphi) = bridge_sde(g, v, theano.gradient.disconnected_grad(w))[:5] # we don't take gradients of the sampling scheme
            return (log_varphi[-1], w)

        (cout, updates) = theano.scan(fn=bridge_logvarphis,
                                      outputs_info=[constant(0.),init_chain if init_chain is not None else T.zeros_like(dWs)],
                                      sequences=[dWsi])
        log_varphi = T.log(T.mean(T.exp(cout[0])))
        log_p_T = -.5 * g[0].shape[0] * T.log(2. * np.pi * Tend) - Cgv / (2. * Tend)# + log_varphi
        p_T = T.exp(log_p_T)
    else:
        # update v during sampling, e.g. for fiber densities
        assert(chain_sampler is None)
        print("transition density with v updates")

        # bridges
        def bridge_p_T(dWs, lp_T, lv):
            Cgv = T.sum(phi(g, lv) ** 2)
            (ts, gs, log_likelihood, log_varphi) = bridge_sde(g, lv, dWs)[:4]
            lp_T =  T.power(2.*np.pi*Tend,-.5*g[0].shape[0])*T.exp(-Cgv/(2.*Tend))#*T.exp(log_varphi[-1])
            lv = options['update_v'](lv)                        
            return (lp_T, lv)

        (cout, updates) = theano.scan(fn=bridge_p_T,
                                      outputs_info=[constant(0.), v],
                                      sequences=[dWsi])
        p_T = T.mean(cout[:][0])
        log_p_T = T.log(p_T)
        v = cout[-1][1]
    
    if chain_sampler is None:
        return (p_T,log_p_T,v)
    else:
        return (p_T,log_p_T,v,w)

# densities wrt. the Riemannian volume form
def p_T(*args,**kwargs): return p_T_log_p_T(*args,**kwargs)[0]
def log_p_T(*args,**kwargs): return p_T_log_p_T(*args,**kwargs)[1]

def dp_T(thetas,*args,**kwargs):
    """ Monte Carlo approximation of transition density gradient """
    lp_T = p_T(*args,**kwargs)
    return (lp_T,)+tuple(T.grad(lp_T,theta) for theta in thetas)

def dlog_p_T(thetas,*args,**kwargs):
    """ Monte Carlo approximation of log transition density gradient """
    llog_p_T = log_p_T(*args,**kwargs)
    return (llog_p_T,)+tuple(T.grad(llog_p_T,theta) for theta in thetas)


In [ ]:
# # Delyon/Hu guided process
from src.stochastics.guided_process import *

# guide function
def phi(u,v):
    x = (u[0][0:M.dim],u[1])
    nu = u[0][M.dim:].reshape((M.dim,-1))
    
    return T.nlinalg.tensorsolve(nu,M.StdLog(x,v).flatten()).reshape((M.dim,))

(stochastic_development_guided,stochastic_development_guidedf) = get_guided_likelihood(
    M,M.sde_development,phi,lambda x: T.eye(M.dim),A=lambda x,v,w: T.dot(v,w),
    use_charts=True,chart_update=M.chart_update_FM)

n_steps.set_value(1000)

w = M.Ff(M.Expf(x,tensor(np.array([.8,-.5]))))
(ts,us,charts,log_likelihood,log_varphi) = stochastic_development_guidedf(u,w,dWsf(M.dim.eval()))[:5]
print("log likelihood: ", log_likelihood[-1], ", log varphi: ", log_varphi[-1])

# plot
newfig()
M.plot()
M.plot_path(zip(us,charts),v_steps=np.arange(0,n_steps.eval(),50))
M.plotx(x,color='r',s=150)
M.plotx(w,color='k',s=150)
plt.show()

n_steps.set_value(100)

In [ ]:
options = {}
options['samples_per_obs'] = 1

# transition density etc.
q0 = M.sym_coords()
v = M.sym_coords()
chart = M.sym_chart()
thetas = (q0,)
_log_p_Tf = theano.function([q0,chart,v],log_p_T((q0,chart),v,dWs(M.dim),stochastic_development_guided,phi,options,sde=M.sde_development,F=M.F,use_charts=True))
# _dlog_p_Tf = theano.function([q0,chart,v],dlog_p_T(thetas,(q0,chart),v,dWs(M.dim),stochastic_development_guided,phi,options,sde=M.sde_development,F=M.F,use_charts=True))
_p_Tf = theano.function([q0,chart,v],T.exp(log_p_T((q0,chart),v,dWs(M.dim),stochastic_development_guided,phi,options,sde=M.sde_development,F=M.F,use_charts=True)))
log_p_Tf = lambda x,v: _log_p_Tf(x[0],x[1],v)
# dlog_p_Tf = lambda x,v: _dlog_p_Tf(x[0],x[1],v)
p_Tf = lambda x,v: _p_Tf(x[0],x[1],v)

v = x
print(x)
print(v)
%time print(log_p_Tf(u,v[0]))
# %time print(p_Tf(x,v[0]))
# %time print(dlog_p_Tf(x,v[0]+.1*np.random.randn(M.dim.eval())))

In [ ]:
pts = 20
phi, theta = np.meshgrid(0.,np.linspace(np.pi/2,-np.pi/2-1e-2,pts))
phitheta = np.vstack([phi.ravel(), theta.ravel()]).T
xs = np.apply_along_axis(M.F_sphericalf,1,phitheta)

# plot points
newfig()
M.plot()
M.plotx(x,color='r',s=100)
for i in range(xs.shape[0]):
    M.plotx((M.invFf((xs[i],x[1])),x[1]))

nu1 = np.dot(np.diag((1.,1.)),np.linalg.cholesky(M.gsharpf(x)))
u1 = (np.concatenate((x[0],nu.flatten())),x[1])
    
# compute transition density for different T
newfig2d()
for t in np.array([.5,1.,2.]):
    print(t)
    Tend.set_value(t)

    fs = np.apply_along_axis(lambda v: p_Tf(u1,M.invFf((v,x[1]))),1,xs)
    print(fs)

    plt.plot(np.pi/2-theta,fs)
Tend.set_value(1.)

# Anisotropic  Normal Distribution

In [ ]:
# plot sample data with trajectories
K = 8
obss = np.zeros((K,n_steps.eval(),M.dim.eval()))
obs_charts = np.zeros((K,n_steps.eval(),)+x[1].shape)
# srng.seed(422)
i = 0
while i < K:
    try:
        (ts,us,charts) = M.stochastic_developmentf(u,dWsf(M.dim.eval()))
        obss[i] = us[:,0:M.dim.eval()]
        obs_charts[i] = charts
        i += 1
    except np.linalg.linalg.LinAlgError:
        pass

# plot samples
colormap = plt.get_cmap('winter')
colors=[colormap(k) for k in np.linspace(0, 1, K)]
newfig()
M.plot()
M.plotx(x,v=u[0][M.dim.eval():].reshape((M.dim.eval(),-1)))
for i in range(K):
    M.plot_path(zip(obss[i],obs_charts[i]),linewidth=.5,color=colors[i])
plt.show()

In [ ]:
# sample data
K = 1024
obss = np.zeros((K,M.dim.eval()))
obs_charts = np.zeros((K,)+x[1].shape)
# srng.seed(422)
i = 0
while i < K:
    try:
        (ts,us,charts) = M.stochastic_developmentf(u,dWsf(M.dim.eval()))
        obss[i] = us[-1][0:M.dim.eval()]
        obs_charts[i] = charts[-1]
        i += 1
    except np.linalg.linalg.LinAlgError:
        pass

# plot samples
newfig()
M.plot()
M.plotx(x,v=u[0][M.dim.eval():].reshape((M.dim.eval(),-1)))
for i in range(K):
    M.plotx((obss[i],obs_charts[i]))
plt.show()

In [ ]:
# # plot estimated density, 
# newfig()
# # plotM(alpha=.4)
# # plot_sphere_density_estimate(M, np.array([M.Ff(obs) for obs in obss]),pts=100,alpha=.8,bandwidth=.15) # spherical coordinates
# plot_density_estimate(M,np.array([M.Ff((obs,chart)) for (obs,chart) in zip(obss,obs_charts)]),limits=[-3,3,-3,3],pts=500,alpha=.4,bandwidth=.15) # general ellipsoidal coordinates (note: very long computation time)
# plt.show()

# Most Probable Paths

In [ ]:
def initialize(M):
    y = M.sym_element()
    y_chart = M.sym_chart()
    p = M.sym_covector()
    
    def loss(u,p,y):
        d = y[0].shape[0]
        (u1,chart1) = M.Exp_Hamiltonian_FM(u,p)
        y_chart1 = M.update_coords(y,chart1)
        return 1./d*T.sum(T.sqr(u1[0:d] - y_chart1[0]))
    M.lossf = M.coords_function(lambda u,p,y,y_chart: loss(u,p,(y,y_chart)),p,y,y_chart)

    def Log_FM(u,y):
        res = minimize(lambda p: M.lossf(u,p,y[0],y[1]), np.zeros(u[0].shape), 
                       method='CG', jac=False, options={'disp': False, 
                                                        'maxiter': 50})
        return res.x
    M.Log_FM = Log_FM
initialize(M)

In [ ]:
# Compute 'most probable path' (in the sense of the driving semi-martingale) between u and x2
x2 = M.coordsf([0.25,1.])

# cotangent vector for the MPP:
px2 = M.Log_FM(u,x2)

# MPP from u to x2:
(us,charts) = M.Exp_Hamiltonian_FMtf(u,px2)

# plot
newfig()
M.plot(rotate=(30,80))
M.plotx(x,color="blue")
M.plotx(x2,color="red")
M.plot_path(zip(us,charts),v_steps=np.arange(0,n_steps.eval(),5),linewidth=1.5,s=50)
plt.show()

# Horizontal Vector Fields

In [ ]:
def plotHorizontal(u,color='b',color_intensity=1.,linewidth=3.,prevx=None,last=True):
        chart = u[1]    
        x = (u[0][0:M.dim.eval()],chart)
        nu = u[0][M.dim.eval():].reshape((M.dim.eval(),-1))
        xM = M.Ff(x)
        
        ax = plt.gca()
        
        # plot frame and horizontal variation
        M.plotx(x)
        Hu = M.Horizontalf(u) # horizontal basis fields
        print("Hu:",Hu)
        Hnu = Hu[M.dim.eval():].reshape((M.dim.eval(),nu.shape[1],nu.shape[1])) # nu part
        JFx = M.JFf(x)
        for j in range(M.dim.eval()):
            nujM = np.dot(JFx,nu[:,j])
            HnujM = np.dot(JFx,np.dot(Hnu,nu[:,j]))
            ax.quiver(xM[0],xM[1],xM[2],nujM[0],nujM[1],nujM[2], pivot='tail',
                      arrow_length_ratio = 0.15, linewidths=1,
                      color='black',normalize=True,length=np.linalg.norm(nujM))
            for k in range(nu.shape[1]):
                basep = xM + nujM
                ax.quiver(basep[0],basep[1],basep[2],
                          HnujM[0,k],HnujM[1,k],HnujM[2,k], pivot='tail',linewidths=2.,
                          color='red',normalize=True,length=0.3)


# plot horizontal vector fields
newfig()
M.plot()
plotHorizontal(u)
plt.show()